## Ejercicio 2: Optimización de Inventario
### Contexto: En Farmacia Saludable, quieren reducir desperdicios analizando compras conjuntas:
#### Tareas:
#### 1. Aplica Apriori con min_support=0.3.
#### 2. Genera reglas con min_confidence=0.5.
#### Preguntas clave: 
#### Si un cliente compra Jarabe, ¿qué otro producto tiene mayor probabilidad de comprar?
#### ¿Qué par de productos debería colocarse juntos en el estante? Justifica con lift.

In [1]:
#Importamos las librerias
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules 
from mlxtend.preprocessing import TransactionEncoder 

In [2]:
transacciones = [
['Analgésico', 'Jarabe', 'Curitas'],
['Termómetro', 'Jarabe'],
['Analgésico', 'Termómetro', 'Alcohol'],
['Jarabe', 'Alcohol', 'Curitas'],
['Analgésico', 'Jarabe'],
['Termómetro', 'Curitas'],
['Analgésico', 'Alcohol']
]

print("Dataset original")
for i, trans in enumerate(transacciones, 1):
    print(f"Transacción {i}: {trans}")

Dataset original
Transacción 1: ['Analgésico', 'Jarabe', 'Curitas']
Transacción 2: ['Termómetro', 'Jarabe']
Transacción 3: ['Analgésico', 'Termómetro', 'Alcohol']
Transacción 4: ['Jarabe', 'Alcohol', 'Curitas']
Transacción 5: ['Analgésico', 'Jarabe']
Transacción 6: ['Termómetro', 'Curitas']
Transacción 7: ['Analgésico', 'Alcohol']


In [3]:
te = TransactionEncoder()
te_ary = te.fit(transacciones).transform(transacciones)
df = pd.DataFrame(te_ary, columns=te.columns_)
print("\nDataFrame transformado (One-Hot Encoding): ")
print(df)


DataFrame transformado (One-Hot Encoding): 
   Alcohol  Analgésico  Curitas  Jarabe  Termómetro
0    False        True     True    True       False
1    False       False    False    True        True
2     True        True    False   False        True
3     True       False     True    True       False
4    False        True    False    True       False
5    False       False     True   False        True
6     True        True    False   False       False


In [4]:
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)

print("\nItemsets frecuentes (soporte >= %30):")
print(frequent_itemsets.sort_values('support', ascending=False))


Itemsets frecuentes (soporte >= %30):
    support      itemsets
1  0.571429  (Analgésico)
3  0.571429      (Jarabe)
0  0.428571     (Alcohol)
2  0.428571     (Curitas)
4  0.428571  (Termómetro)


In [12]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
print("\nReglas de asociación (confianza >= 50% sin lift > 1):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False))


Reglas de asociación (confianza >= 50% sin lift > 1):
Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


#### Pocas combinaciones de productos ocurren con una frecuencia del 30% o más, usare 0.2 para ampliar el alcance del analisis

In [14]:
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)

print("\nItemsets frecuentes (soporte >= %20):")
print(frequent_itemsets.sort_values('support', ascending=False))


Itemsets frecuentes (soporte >= %20):
    support               itemsets
1  0.571429           (Analgésico)
3  0.571429               (Jarabe)
0  0.428571              (Alcohol)
2  0.428571              (Curitas)
4  0.428571           (Termómetro)
5  0.285714  (Alcohol, Analgésico)
6  0.285714   (Analgésico, Jarabe)
7  0.285714      (Curitas, Jarabe)


In [15]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
print("\nReglas de asociación (confianza >= 50% sin lift > 1):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False))


Reglas de asociación (confianza >= 50% sin lift > 1):
    antecedents   consequents   support  confidence      lift
0     (Alcohol)  (Analgésico)  0.285714    0.666667  1.166667
1  (Analgésico)     (Alcohol)  0.285714    0.500000  1.166667
5      (Jarabe)     (Curitas)  0.285714    0.500000  1.166667
4     (Curitas)      (Jarabe)  0.285714    0.666667  1.166667
3      (Jarabe)  (Analgésico)  0.285714    0.500000  0.875000
2  (Analgésico)      (Jarabe)  0.285714    0.500000  0.875000


#### Si un cliente compra Jarabe, ¿qué otro producto tiene mayor probabilidad de comprar?

In [ ]:
#"Hay una tendencia significativa a que los clientes que compran Jarabe también compren Curitas. 
# Esta combinación ocurre en el 28.6% de las compras y es un 16.6% más frecuente"

In [16]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules = rules[rules['lift'] > 1]
print("\nReglas de asociación (confianza >= 50% con lift > 1):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False))


Reglas de asociación (confianza >= 50% con lift > 1):
    antecedents   consequents   support  confidence      lift
0     (Alcohol)  (Analgésico)  0.285714    0.666667  1.166667
1  (Analgésico)     (Alcohol)  0.285714    0.500000  1.166667
4     (Curitas)      (Jarabe)  0.285714    0.666667  1.166667
5      (Jarabe)     (Curitas)  0.285714    0.500000  1.166667


#### ¿Qué par de productos debería colocarse juntos en el estante? Justifica con lift.

In [ ]:
#La combinación Jarabe – Curitas tiene un lift de 1.1667, lo que significa que se compran juntos un 16.67% 